In [1]:
!pip install torchsummary

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.Dropout(0.05),
            nn.BatchNorm2d(10),
            
            nn.ReLU()
        ) # output_size = 24
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=14, kernel_size=(3, 3), padding=0, bias=False),
            
            nn.BatchNorm2d(14),
            nn.ReLU()
        ) # output_size = 22

        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 11
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            nn.Dropout(0.05),
            nn.BatchNorm2d(10),
            
            nn.ReLU()
        ) # output_size = 11

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            
            nn.ReLU()
        ) # output_size = 9
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=14, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(14),
            nn.ReLU()
        ) # output_size = 7
        self.pool2 = nn.MaxPool2d(2, 2) 
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 7
        self.gap = m = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        # print(x.shape)
        x = self.gap(x)
        x = x.view(-1, 10)
        # print(x.shape)
        return F.log_softmax(x, dim=-1)

In [4]:


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             900
           Dropout-5           [-1, 10, 24, 24]               0
       BatchNorm2d-6           [-1, 10, 24, 24]              20
              ReLU-7           [-1, 10, 24, 24]               0
            Conv2d-8           [-1, 14, 22, 22]           1,260
       BatchNorm2d-9           [-1, 14, 22, 22]              28
             ReLU-10           [-1, 14, 22, 22]               0
        MaxPool2d-11           [-1, 14, 11, 11]               0
           Conv2d-12           [-1, 10, 11, 11]             140
          Dropout-13           [-1, 10, 11, 11]               0
      BatchNorm2d-14           [-1, 10,

In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
def train(model, device, train_loader, optimizer, epoch):
    average_epoch_loss = 0
    correct_predictions_epoch = 0
    model.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        for batch_idx, (data, target) in enumerate(tepoch):
            tepoch.set_description(f"Train Epoch {epoch}")
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            average_epoch_loss += loss.item()
            correct_predictions = sum(output.argmax(dim = 1) == target)
            correct_predictions_epoch += correct_predictions
            tepoch.set_postfix(loss = round(loss.item(),5), accuracy = format(100 *correct_predictions.item()/data.shape[0], '.4f'))
    return average_epoch_loss/batch_idx,correct_predictions_epoch/len(train_loader.dataset)

In [7]:
from tqdm.notebook import tqdm

In [8]:
def test(model, device, test_loader):
    average_epoch_loss = 0
    correct_predictions_epoch = 0
    model.eval()
    with torch.no_grad():
        with tqdm(test_loader, unit="batch") as tepoch:
            for batch_idx, (data, target) in enumerate(tepoch):
                tepoch.set_description(f"Test  Epoch {epoch}")
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = F.nll_loss(output, target)  # sum up batch loss
                average_epoch_loss += loss.item()
                correct_predictions = sum(output.argmax(dim = 1) == target)  # get the index of the max log-probability
                correct_predictions_epoch += correct_predictions
                tepoch.set_postfix(loss = round(loss.item(),5), accuracy = format(100 *correct_predictions.item()/data.shape[0], '.4f'))
    return average_epoch_loss/batch_idx,correct_predictions_epoch/len(test_loader.dataset)

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
train_loss = []
test_loss = []

train_accuracy = []
test_accuracy = []
for epoch in range(1, 15):

    tr_loss,tr_acc = train(model, device, train_loader, optimizer, epoch)
    tst_loss,tst_acc = test(model, device, test_loader)
    train_loss.append(tr_loss),train_accuracy.append(tr_acc)
    test_loss.append(tst_loss),test_accuracy.append(tst_acc)
    print("Train_epoch : ",100*tr_acc.item())
    print("Test_epoch : ",100*tst_acc.item())



Train_epoch :  88.46166729927063
Test_epoch :  94.43999528884888



Train_epoch :  97.5266695022583
Test_epoch :  97.46999740600586



Train_epoch :  97.96833395957947
Test_epoch :  98.29999804496765



Train_epoch :  98.28833341598511
Test_epoch :  97.92999625205994



Train_epoch :  98.45166802406311
Test_epoch :  98.19999933242798


...


Train_epoch :  98.70166778564453
Test_epoch :  98.60000014305115



Train_epoch :  98.79833459854126
Test_epoch :  98.3299970626831



Train_epoch :  98.84333610534668
Test_epoch :  98.89999628067017



Train_epoch :  98.9733338356018
Test_epoch :  98.69999885559082



Train_epoch :  98.96166920661926
Test_epoch :  98.69999885559082
